<a href="https://colab.research.google.com/github/aanchal0431/chatbot/blob/main/SEP_728_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [39]:
print(tf.__version__)

2.6.0


Git Commands to clone repository, pull and push data

In [48]:
#!git clone https://github.com/aanchal0431/chatbot.git
# !git pull
#%cd chatbot/
#!git config --global user.name "aanchal0431"
#!git config --global user.email "aanchal0431@gmail.com"
%ls


Data/  README.md  SEP_728_Chatbot.ipynb


Load Datasets

In [49]:
cur_path = 'Data/Question_Answer_Dataset_v1.2/'
data_s8 = pd.read_csv(cur_path + 'S08/question_answer_pairs.txt', delimiter="\t")
print("Shape:", data_s8.shape)

Shape: (1715, 6)
